In [187]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup as soup
from tqdm import tqdm_notebook as tqdm
from time import sleep

# Set up Page

In [188]:
PAGE_URL = 'https://www.rusprofile.ru/codes/260000'

In [189]:
def get_soup(url):
    """Makes tasty soup out of raw HTML"""
    return soup(requests.get(url).text)

# Get Number of Pages

In [190]:
base_page_soup = get_soup(PAGE_URL) # get page soup to extract n of pages
pg_list = base_page_soup.find('ul', class_="paging-list") # find pagination item on the page
N_PAGES = int(pg_list.find_all('li')[-2].text)
iterator_arg = range(1, N_PAGES + 1) # define iterator for browsing pages
print(f'Total pages to scrape from: {N_PAGES}')

Total pages to scrape from: 145


# Scapping functions

In [191]:
def get_companies(page_soup):
    """Finds companies on the page. Returns tags list"""
    return page_soup.find_all('div', class_ = 'company-item')

In [192]:
month_dict = {
    'января' : 1, 'февраля' : 2, 'марта' : 3, 'апреля' : 4, 'мая' : 5, 'июня' : 6,
    'июля' : 7, 'августа' : 8, 'сентября' : 9, 'октября' : 10, 'ноября' : 11, 'декабря' : 12
} # dictionary for transferring russian names into integer months numbers

In [199]:
def scrape_company(company):
    """Extracts data from each company item"""
    company_name = company.find('div', class_="company-item__title").find('a').text.strip()
    try:
        company_status = company.find('span', class_="warning-text").text.strip()
    except:
        company_status = 'Организация работает'
    company_address = ', '.join(company.find('address',  class_="company-item__text").text.strip().split(', ')[1:3])
    lst = company.findAll('div', class_='company-item-info')[1].findAll('dd')[2:]
    if len(lst) == 2:
        [company_estdate, company_cap] =  [i.text.strip() for i in lst]
        company_estdate = company_estdate.strip(' г.').split(' ')
        company_estdate = pd.Timestamp(f'{company_estdate[2]}-{month_dict[company_estdate[1]]}-{company_estdate[0]}')
        company_cap = pd.to_numeric(company_cap.strip(' руб.').replace(' ', '').replace(',', '.'))
    elif len(lst) == 1:
        company_estdate = lst[0].text.strip().strip(' г.').split(' ')
        company_estdate = pd.Timestamp(f'{company_estdate[2]}-{month_dict[company_estdate[1]]}-{company_estdate[0]}')
        company_cap = None
    else:
        company_estdate, company_cap = None, None
    company_activity = company.findAll('div',  class_="company-item-info")[-1].find('dd').text.strip()
    return pd.Series({'name' : company_name, 'status' : company_status,
                      'address' : company_address, 'est_date' : company_estdate,
                      'cap' : company_cap, 'activity' : company_activity})

# Scraping

In [194]:
page_url_iterable = PAGE_URL + '/{}/' # Making the string changable

In [206]:
cat_num = PAGE_URL.split('/')[-1]

In [ ]:
SLEEP_TIME = 5 # Essential! Could be increased manually if needed. Defines pause betweeen opening the pages

In [ ]:
df_list = []
for page_n in tqdm(iterator_arg):
    current_page_soup = get_soup(page_url_iterable.format(page_n))
    sleep(SLEEP_TIME)
    companies = get_companies(current_page_soup)
    current_page_df = pd.DataFrame([scrape_company(company) for company in companies])
    df_list.append(current_page_df)
    
full_cat_df = pd.concat(df_list)
full_cat_df = full_cat_df.assign(base_category = int(cat_num))
full_cat_df = full_cat_df[full_cat_df.activity.str.contains(cat_num[:2])]

In [209]:
full_cat_df.head()

,name,status,address,est_date,cap,activity,base_category
0,"ООО ""Феррит-Сервис""",Организация ликвидирована,"Астраханская область, город Астрахань",2004-07-21,15380.0,"26 Производство компьютеров, электронных и опт...",260000
1,"ООО НПФ""Элепс""",Организация работает,"республика Татарстан, город Казань",2001-11-30,700000.0,"26.60.4 Производство инструмента, оборудования...",260000
2,"ПАО ""Нептун""",Организация работает,"Ставропольский край, город Ставрополь",1994-04-11,62420.5,26.51.7 Производство приборов и аппаратуры для...,260000
3,"АО ""Экон""",Организация работает,"Калужская область, город Обнинск",1997-01-15,10000.0,26.51.5 Производство приборов для контроля про...,260000
4,"АО ""АПЗ""",Организация работает,"Нижегородская область, город Арзамас",1993-09-06,192636600.0,"26.51.6 Производство прочих приборов, датчиков...",260000


In [211]:
full_cat_df.to_csv(f'data/database_{cat_num}.csv', index = None, encoding = 'utf-8')